In [55]:
import json
import os

import devlib
from wlgen import RTA

import trappy

In [56]:
# Configure logging for this session
import logging
reload(logging)
logging.basicConfig(
    format='%(asctime)-9s %(levelname)-8s: %(message)s',
    level=logging.DEBUG,
    datefmt='%I:%M:%S')

# Global configuration

In [57]:
# Host side results folder
RESULTS_DIR = '/tmp/schedtest'

# Taerget side temporary folder
TARGET_DIR = '/root/schedtest'

# List of tools to install on the target system
TOOLS = ["rt-app", "trace-cmd", "taskset"]

# FTrace Configuration
FTRACE_EVENTS = ['sched_switch', 'cpu_frequency']
FTRACE_BUFFZISE = 10240

# HWMon Configuration
HWMON_CONF = {
    'sites' : [ 'a53', 'a57' ],
    'kinds' : [ 'energy' ]
}

# List of modules to enable
MODULES = ['bl', 'cpufreq', 'hwmon']

# Target configuration

In [58]:
target = devlib.LinuxTarget(
    connection_settings={
        'host'     : '192.168.0.10',
        'username' : 'root',
        'password' : ''
    },
    load_default_modules=False,
    modules=MODULES,
    working_directory=TARGET_DIR
)

12:01:16  DEBUG   : Installing module bl
12:01:18  DEBUG   : Installing module cpufreq
12:01:18  DEBUG   : Installing module hwmon


In [59]:
logging.info("Target ABI: %s, CPus: %s",
             target.abi,
             target.cpuinfo.cpu_names)

12:01:45  INFO    : Target ABI: arm64, CPus: ['A53', 'A57', 'A57', 'A53', 'A53', 'A53']


In [60]:
logging.info('Copy tools required on target')
tools_to_install = []
for tool in TOOLS:
    # deploy scripts or...
    binary = '../../tools/scripts/{}'.format(tool)
    if not os.path.isfile(binary):
        # ... binary tool matching the target ABI
        binary = '../../tools/{}/{}'.format(target.abi, tool)
    tools_to_install.append(binary)
target.setup(tools_to_install)

12:01:45  INFO    : Copy tools required on target


# FTrace configuration

In [61]:
ftrace = None
if len(FTRACE_EVENTS):
    ftrace = devlib.FtraceCollector(
        target,
        events = FTRACE_EVENTS,
        buffer_size = FTRACE_BUFFZISE,
        autoreport = False,
        autoview = False
    )
    logging.info('FTrace configured to collect events:')
    logging.info('  %s', FTRACE_EVENTS)
else:
    logging.info('FTrace collection disabled by configuration')
    
def ftrace_start():
    if ftrace is None:
        return
    ftrace.start()
    logging.info('FTrace STARTED')
    
def ftrace_stop():
    if ftrace is None:
        return
    ftrace.stop()
    logging.info('FTrace STOPPED')
    ftrace.get_trace(RESULTS_DIR + '/trace.dat')

12:01:50  INFO    : FTrace configured to collect events:
12:01:50  INFO    :   ['sched_switch', 'cpu_frequency']


# Energy Meter configuration

In [62]:
hwmon = None
if 'hwmon' in MODULES:
    hwmon = devlib.HwmonInstrument(target)
    hwmon.reset(**HWMON_CONF)
    logging.info('Channels selected for energy sampling:')
    logging.info('  %s', str(hwmon.active_channels))
else:
    logging.info('Energy sampling disabled by configuration')
    
# The contained for sampled energy values
energy_reading = {}

def hwmon_reset():
    energy_reading = {}

# Simple function to compute energy-deltas among consecutive readings
def hwmon_sample():
    if hwmon is None:
        return
    samples = hwmon.take_measurement()
    logging.debug('Measure: %s', samples)
    for s in samples:
        label = s.channel.label\
                    .replace('_energy', '')\
                    .replace(" ", "_")
        value = s.value
        logging.debug('Update %s: %s',
                      label, value)

        if label not in energy_reading:
            energy_reading[label] = {
                 'last'  : value,
                 'delta' : 0,
                 'total' : 0
                }
            logging.debug('Initialize %s: %s',
                          label, energy_reading[label])
            continue

        last  = energy_reading[label]['last']
        delta = value - last
        total = energy_reading[label]['total']

        energy_reading[label]['last']  = value
        energy_reading[label]['delta'] = delta
        energy_reading[label]['total'] = total + delta
        
        logging.debug('Update %s: %s',
                          label, energy_reading[label])

    return energy_reading


12:01:50  DEBUG   : Discovering available HWMON sensors...
12:01:50  DEBUG   : 	Adding sensor v2m_juno_amp/curr1
12:01:50  DEBUG   : 	Adding sensor v2m_juno_amp/curr1
12:01:50  DEBUG   : 	Adding sensor v2m_juno_power/power1
12:01:50  DEBUG   : 	Adding sensor v2m_juno_power/power1
12:01:50  DEBUG   : 	Adding sensor v2m_juno_energy/energy1
12:01:50  DEBUG   : 	Adding sensor v2m_juno_energy/energy1
12:01:50  DEBUG   : 	Adding sensor v2m_juno_energy/energy1
12:01:50  DEBUG   : 	Adding sensor v2m_juno_energy/energy1
12:01:50  DEBUG   : 	Adding sensor v2m_juno_amp/curr1
12:01:50  DEBUG   : 	Adding sensor v2m_juno_amp/curr1
12:01:50  DEBUG   : 	Adding sensor v2m_juno_volt/in1
12:01:50  DEBUG   : 	Adding sensor v2m_juno_volt/in1
12:01:50  DEBUG   : 	Adding sensor v2m_juno_volt/in1
12:01:50  DEBUG   : 	Adding sensor v2m_juno_volt/in1
12:01:50  DEBUG   : 	Adding sensor v2m_juno_power/power1
12:01:50  DEBUG   : 	Adding sensor v2m_juno_power/power1
12:01:50  INFO    : Channels selected for energy 

# Workload configuration

In [63]:
# RTApp calibration

# Uncomment the following line to calibrate RTApp the first time
# target_calibration = RTA.calibrate(target)

# Provide a pre-defined calibration
target_calibration = {0: 353, 1: 138, 2: 138, 3: 353, 4: 354, 5: 360}

In [64]:
rtapp = RTA(target, 'simple', calibration=target_calibration)
rtapp.conf(
    kind='profile',
    params={
        'task_p20': RTA.periodic(
            period_ms=100,
            duty_cycle_pct=20,
            duration_s=5,
            # Run this task on the first CPU of the target
            cpus='0'),
        'task_r20_5-60': RTA.ramp(
            start_pct=5,
            end_pct=65,
            delta_pct=20,
            # Run this task on the last CPU of the target
            cpus=str(len(target.core_names)-1)),
    },
    run_dir=TARGET_DIR
);

12:01:50  ERROR   : Assuming taskset is preinstalled
12:01:50  INFO    : Setup new workload simple
12:01:50  DEBUG   : Setup step [postrun] callback to [__postrun] function
12:01:50  DEBUG   : Configuring a profile-based workload...
12:01:50  DEBUG   : ref on big cpu: 1
12:01:50  INFO    : Workload duration defined by longest task
12:01:50  INFO    : Default policy: SCHED_OTHER
12:01:50  INFO    : ------------------------
12:01:50  INFO    : task [task_p20], sched: using default policy
12:01:50  INFO    :  | loops count: 1
12:01:50  INFO    :  | CPUs affinity: 0
12:01:50  INFO    :  + phase_000001: duration 5.000000 [s] (50 loops)
12:01:50  INFO    :  |  period   100000 [us], duty_cycle  20 %
12:01:50  INFO    :  |  run_time  20000 [us], sleep_time  80000 [us]
12:01:50  INFO    : ------------------------
12:01:50  INFO    : task [task_r20_5-60], sched: using default policy
12:01:50  INFO    :  | loops count: 1
12:01:50  INFO    :  | CPUs affinity: 5
12:01:50  INFO    :  + phase_000001:

# Workload execution

In [65]:
logging.info('Create output folder')
os.system('mkdir {}'.format(RESULTS_DIR));

12:01:50  INFO    : Create output folder


In [66]:
hwmon_reset()

ftrace_start()
hwmon_sample()

logging.info('RTApp STARTING...')
rtapp.run(out_dir=RESULTS_DIR)

nrg = hwmon_sample();
ftrace_stop()

12:01:57  INFO    : FTrace STARTED
12:01:57  DEBUG   : Measure: [a53_energy: 9493.324255 joules, a57_energy: 13568.999657 joules]
12:01:57  DEBUG   : Update a53: 9493.324255
12:01:57  DEBUG   : Initialize a53: {'total': 0, 'last': 9493.324255, 'delta': 0}
12:01:57  DEBUG   : Update a57: 13568.999657
12:01:57  DEBUG   : Initialize a57: {'total': 0, 'last': 13568.999657, 'delta': 0}
12:01:57  INFO    : RTApp STARTING...
12:01:57  INFO    : Executor [start]: /root/schedtest/bin/rt-app /root/schedtest/simple_00.json
12:02:03  DEBUG   : Callback [postrun]...
12:02:03  DEBUG   : Pulling logfiles to [/tmp/schedtest]...
12:02:03  DEBUG   : Pulling JSON to [/tmp/schedtest]...
12:02:03  DEBUG   : Saving output on [/tmp/schedtest/output.log]...
12:02:03  INFO    : Executor [end]: /root/schedtest/bin/rt-app /root/schedtest/simple_00.json
12:02:04  DEBUG   : Measure: [a53_energy: 9503.099997 joules, a57_energy: 13571.409729 joules]
12:02:04  DEBUG   : Update a53: 9503.099997
12:02:04  DEBUG   : Upd

In [67]:
logging.info('Energy: %s',
    json.dumps(nrg,
                indent=4,
                separators=(',', ': ')))

12:02:08  INFO    : Energy: {
    "a53": {
        "total": 9.77574199999981,
        "last": 9503.099997,
        "delta": 9.77574199999981
    },
    "a57": {
        "total": 2.410072000000582,
        "last": 13571.409729,
        "delta": 2.410072000000582
    }
}


# Trace inspection

In [68]:
trappy.plotter.plot_trace(RESULTS_DIR)